In [67]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os 

load_dotenv()

langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

In [17]:
loader = PyPDFLoader("./files/kafka_guide.pdf")
loader2 = UnstructuredFileLoader("./files/kafka_guide.pdf")

In [18]:
loader.load()
# loader2.load()

[Document(page_content='신뢰성  있게  카프카를  사용하는  방법\n1신뢰성  있게  카프카를  사용하는  방법\nG o a l  \nAp ache Kafka 를  신뢰성  있게  사용하는  방법에  대해  알아보자 . \n카프카에  대한  기본적인  내용  \n앞으로  설명할  내용은  카프카에  대한  기본  지식을  가정하에  진행되므로 , 카프카를  처음  접한\n다면  아래의  코스를  통해  미리  학습할  것을  권장합니다 . \nS h o r t  C o u r s e  ( K o r e a n )  \nKafka 조금  아는  척하기  시리즈  1\nKafka 조금  아는  척하기  시리즈  2 (Pr oducer)\nKafka 조금  아는  척하기  시리즈  3 (Consumer)  \nS h o r t  C o u r s e  ( E n g l i s h )  \nConfluent Kafka 101 Course  \n카프카가  기본적으로  제공해주는  신뢰성  \n데이터베이스  트랜잭션은  ‘A CIDʼ 기능을  신뢰성  있게  제공한다 . 마찬가지로  카프카  역시  기\n본적으로  신뢰성  있게  제공하는  기능들이  있다 . \n신뢰성은   “시스템이  예상한대로  올바르게  작동하는  것 ˮ  을  말한다 . 카프카에게  원하는  신뢰\n성은  아마도  보낸  메시지가  정확히  전달되고 ,  저장되며 ,  처리되는  것을  말할  것이다 . 즉 , 메시\n지가  전달되지  않거나 , 유실되거나 , 갑자기  삭제되거나 , 메시지  처리가  스킵되거나 , 여러번  \n처리되지  않는  것들을  포함할  것이다 . ', metadata={'source': './files/kafka_guide.pdf', 'page': 0}),
 Document(page_content='신뢰성  있게  카프카를  사용하는  방법\n2카프카가  매력적인  이유는  기본적으로  제공하는  신뢰성이  있고 , 개발자가  어떻게  설정하냐\n에

In [19]:
len(loader.load())

11

In [20]:
spliter = RecursiveCharacterTextSplitter()
docs = spliter.split_documents(loader.load())

In [37]:
len(spliter.split_documents(loader.load()))

11

In [42]:
spliter2 = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=100, 
    length_function=len
)

In [68]:
import openai
import os

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [42]:
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import Chroma

cache_dir = LocalFileStore("./.cache/")

embedder = OpenAIEmbeddings()

cached_embedding = CacheBackedEmbeddings.from_bytes_store(embedder, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embedding)

In [40]:
results = vectorstore.similarity_search("카프카를 컨슈머에서 잘 쓰는 방법은?")
len(results)

4

In [41]:
results[0]

Document(page_content='신뢰성  있게  카프카를  사용하는  방법\n2카프카가  매력적인  이유는  기본적으로  제공하는  신뢰성이  있고 , 개발자가  어떻게  설정하냐\n에  따라서  원하는  수준의  신뢰성을  달성할  수  있다 . \n다음은  카프카에서  기본적으로  제공해주는  신뢰성이다 . \n토픽  파티션내의  메시지들은  순서대로  저장되고 , 컨슈머에  의해  순서대로  처리된다 . \n레플리카가  존재하는  한  메시지는  유실되지  않는다 . \n복제의  중요성  \n카프카는  저장된  메시지들을  즉시  복제해서 , 메시지  유실을  방지한다 . 이는  M ySQL 와  같은  \n데이터베이스와는  다른  방식이다 . M ySQL 같은  데이터베이스는  f sync() 같은  시스템  콜을  \n이용해서  데이터를  직접  디스크에  저장한다 . 즉  데이터베이스는  이  방법으로  데이터  유실을  \n방지한다 . \n그러나  카프카는  대량의  데이터  처리를  위해서  바로  디스크에  저장하지  않는다 . Page  \nCache 라는  커널  메모리  공간에서만  데이터를  쓰고 , 이후에  시간이  지나면  데이터는  디스\n크로  반영된다 . 그러므로  디스크에  반영되기  전에  카프카  브로커가  다운된다면  메시지는  유\n실될  수  있다 . \n카프카에서는  이  문제를  복제를  통해  해결한다 . 데이터를  다른  브로커의  Page Cache 에  복\n제  함으로써  브로커가  다운되더라도  데이터가  유실되지  않도록  한다 .', metadata={'page': 1, 'source': './files/kafka_guide.pdf'})

In [60]:
from langchain.chains import RetrievalQA 

llm = ChatOpenAI(
    temperature=0.1
)

retriever = vectorstore.as_retriever()

chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever= retriever
)

chain.run("카프카에서 브로커를 신뢰성있게 쓰려면 어떻게 해야하는가?")

'카프카에서 브로커를 신뢰성 있게 사용하기 위해서는 다음과 같은 설정을 고려해야 합니다:\n1. 토픽 파티션의 복제 값을 정하는 것\n2. 최소 In-Sync 레플리카 수를 유지하는 것\n3. 토픽 파티션의 복제 수를 올바른 값으로 정하기\n\n이러한 설정을 통해 데이터의 가용성과 신뢰성을 높일 수 있습니다.'

In [61]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assitant. answer questions uisng only the following context. If you don't know the answer just say you don't know don't make it up: \n\n{context}"),
    ("human", "{question}")
])
 
 
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm 

chain.invoke("카프카에서 브로커를 신뢰성있게 쓰려면 어떻게 해야하는가?")

AIMessage(content='카프카에서 브로커를 신뢰성 있게 사용하기 위한 설정으로는 다음과 같은 것들이 있습니다:\n1. 토픽 파티션의 복제 값을 정하는 것\n2. 최소 In-Sync 레플리카 수를 유지하는 것\n3. 토픽 파티션의 복제 수를 올바른 값으로 정하는 것\n\n이러한 설정을 통해 브로커가 저장된 메시지들을 안전하게 보관하여 유실되지 않도록 할 수 있습니다.', response_metadata={'token_usage': {'completion_tokens': 160, 'prompt_tokens': 2559, 'total_tokens': 2719}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-d88594fe-e8c9-471c-97a6-1edb95ae49d1-0')

In [65]:
from langchain.schema.runnable import RunnableLambda

map_doc_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. 
    -----
    {context}
    """),
    ("human", "{question}")
])

def map_docs(inputs): 
    documents = inputs["documents"]
    question = inputs["question"]
    results = []
    for document in documents: 
        result = map_doc_chain.invoke({
            "context": document.page_content, 
            "question": question
        }).content 
        results.append(result)
    results = "\n\n".join(results)
    return results

final_prompt = ChatPromptTemplate.from_messages([
    ("system",  """
    Given the following extracted parts of a long document and a question, create a final answer. 
    If you don;t know the answer, just say that you don't know. Don't try to make up an answer.
    ----
    {context} 
    """),
    ("human", "{question}")
])

map_doc_chain = map_doc_prompt | llm 

map_chain =  { "documents": retriever, "question": RunnablePassthrough() } | RunnableLambda(map_docs)   

chain = { "context": map_chain, "question": RunnablePassthrough() } | final_prompt | llm

chain.invoke("카프카 컨슈머를 신뢰성있게 사용하는 방법을 알려줘.")

AIMessage(content='카프카 컨슈머를 신뢰성 있게 사용하기 위해서는 몇 가지 중요한 설정과 방법이 있습니다. 먼저, 컨슈머 그룹을 올바르게 구성하여 메시지를 안전하게 처리할 수 있도록 해야 합니다. 또한 오프셋 관리를 신중하게 해야 하며, 에러 핸들링을 적절히 처리해야 합니다. 마지막으로, 컨슈머의 성능 모니터링과 에러 핸들링을 효과적으로 수행하여 시스템이 예상대로 작동하도록 유지해야 합니다. 이러한 방법들을 준수하면 카프카 컨슈머를 신뢰성 있게 사용할 수 있습니다.', response_metadata={'token_usage': {'completion_tokens': 239, 'prompt_tokens': 1587, 'total_tokens': 1826}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-486b952b-00f3-4b29-9224-9dc340d927bf-0')